In [1]:
# 1. Setup
!pip install --upgrade pip
!pip uninstall -y transformers tokenizers accelerate -q
!pip install "transformers==4.56.0" "protobuf>=5.29.4" -q
!pip install torch datasets -q
!pip install pandas matplotlib seaborn tqdm wandb pyyaml
!pip install bitsandbytes accelerate
!pip install --force-reinstall --no-cache-dir "numpy<2.0"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 20.4 MB/s eta 0:00:0000:010:01
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
peft 0.16.0 requires accelerate>=0.21.0, which is not installed.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
peft 0.16.0 requires accelerate>=0.21.0, which is not installed.
pylibcudf-cu12 25.2.2 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 22.0.0 which is incompatible.
cudf-cu12 25.2.2 requires pyarrow<20.0.0a0,>=14.0.0; pla

In [ ]:
# Clone the latest github repo version
if "google.colab" in str(get_ipython()):
    print("✅ Environment: Google Colab")
    base_data_path = "/content/"
    base_output_path = "/content/output/"
    environment_name = "colab"
elif os.environ.get("KAGGLE_KERNEL_RUN_TYPE"):
    print("✅ Environment: Kaggle")
    base_data_path = "/kaggle/input/"
    base_output_path = "/kaggle/working/"
    environment_name = "kaggle"
else:
    print("⚠️ Environment: Local/Unknown")
    base_data_path = "./data/"
    base_output_path = "./output/"
    environment_name = "local"
    
%cd base_data_path
!rm -r -f OuroTrace
!git clone https://github.com/dzungphieuluuky/OuroTrace.git
%cd OuroTrace

In [ ]:
# Suppress warnings for clean output
import warnings
import os
warnings.filterwarnings("ignore", category=UserWarning)
os.environ["TRANSFORMERS_NO_ADVISORY_WARNINGS"] = "true"
print("✅ Packages installed successfully!")

In [2]:
"Built-in libraries"
import re
import sys
import gc
import time
import json
import hashlib
import glob
import zipfile
from io import StringIO
from datetime import datetime
from typing import Dict, List, Optional, Tuple, Any
import yaml
import logging
import random

"Deep learning and NLP libraries"
import torch
import torch.nn.functional as F
from transformers import (
    AutoConfig, 
    AutoTokenizer, 
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    GenerationConfig,
    logging as hf_logging
)

"Data processing libraries"
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import wandb
from tqdm.auto import tqdm
from IPython import get_ipython

# Configure logging
logging.getLogger("ContinuousBatchingLogger").setLevel(logging.ERROR)
hf_logging.set_verbosity_error()

print(f"Python Version: {sys.version}")
print(f"PyTorch Version: {torch.__version__}")
print(f"CUDA Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA Version: {torch.version.cuda}")
!nvidia-smi

Python Version: 3.11.13 (main, Jun  4 2025, 08:57:29) [GCC 11.4.0]
PyTorch Version: 2.6.0+cu124
CUDA Available: True
CUDA Version: 12.4
Sun Dec 14 09:27:45 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 570.172.08             Driver Version: 570.172.08     CUDA Version: 12.8     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   33C    P8             10W /   70W |       3MiB /  15360MiB |      0%      Default |
|   

In [3]:
import pandas as pd
from src.config_loader import load_config_from_json, post_process_config
from src.runner import run_batch_experiment
from src.evaluation import analyze_experiment_results

# 1. Load Configuration from JSON
config = load_config_from_json('configs/batch_ouro_1.4b_thinking.json')

# 2. Modify settings programmatically (Override JSON defaults)
config['INFERENCE_STEPS'] = [1] 

# 3. Post-process (Convert 'torch.float16' string to object, generate timestamps)
config = post_process_config(config)

# 4. Execute
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
print(f"🕒 Timestamp: {timestamp}")
print("🚀 Starting Experiment...")
acc_results, ppl_results = run_batch_experiment(config)

🚀 Starting Experiment...
🔗 Initializing W&B (timeout: 30s)...
⚠️ W&B failed: 1 validation error for Settings
start_timeout
  Extra inputs are not permitted [type=extra_forbidden, input_value=30, input_type=int]
    For further information visit https://errors.pydantic.dev/2.12/v/extra_forbidden. Continuing offline.
Generating new test datasets...

🧪 EXPERIMENT: UT Steps = 1
Loading model: UT steps=1, Early exit=-1.0


config.json: 0.00B [00:00, ?B/s]

configuration_ouro.py: 0.00B [00:00, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

...

    📊 Acc=20.00% | Time/Sample=18.874s | Total=377.5s

📝 Task: p_hop (20 samples)


Solving 4 requests: 100%|██████████| 4/4 [00:03<00:00,  1.09request/s]


    📊 Acc=35.00% | Time/Sample=0.952s | Total=19.0s

📝 Task: igsm (20 samples)


Solving 2 requests: 100%|██████████| 2/2 [01:56<00:00, 58.24s/request] 


    📊 Acc=40.00% | Time/Sample=58.370s | Total=1167.4s

📊 Analysis Results:
                    is_correct_mean  is_correct_count  is_correct_std  \
ut_steps task_type                                                      
1        igsm                  0.40                20           0.503   
         n_ary                 0.20                20           0.410   
         p_hop                 0.35                20           0.489   

                    generation_time_mean  generation_time_min  \
ut_steps task_type                                              
1        igsm                     58.370               57.980   
         n_ary                    18.874               14.347   
         p_hop                     0.952                0.935   

                    generation_time_max  generated_tokens_mean  
ut_steps task_type                                              
1        igsm                    58.784                 1024.0  
         n_ary                   30.7

In [6]:
# 3. Save Results
df_acc = pd.DataFrame(acc_results)
df_ppl = pd.DataFrame(ppl_results)

RUN_RESULTS_NAME = f"run_{timestamp}"
os.makedirs(os.path.join(OUTPUT_PATH, RUN_RESULTS_NAME))
acc_path = os.path.join(OUTPUT_PATH, RUN_RESULTS_NAME, f"ouro_acc_{timestamp}.csv")
ppl_path = os.path.join(OUTPUT_PATH, RUN_RESULTS_NAME, f"ouro_ppl_{timestamp}.csv")
cfg_path = os.path.join(OUTPUT_PATH, RUN_RESULTS_NAME, f"ouro_config_{timestamp}.yaml")

df_acc.to_csv(acc_path, index=False)
if not df_ppl.empty:
    df_ppl.to_csv(ppl_path, index=False)

# Save Config
def sanitize_config(cfg):
    """Convert config to YAML-safe format"""
    clean = {}
    for k, v in cfg.items():
        if isinstance(v, dict):
            clean[k] = sanitize_config(v)
        elif str(type(v)).find('torch.') != -1:
            clean[k] = str(v)
        else:
            clean[k] = v
    return clean

with open(cfg_path, 'w') as f:
    yaml.dump(sanitize_config(BatchConfig), f)

print(f"\n💾 Results saved to {OUTPUT_PATH}")

# 4. Visualization & Reporting
if not df_acc.empty:
    print("\n" + "="*50 + "\n📊 VISUALIZATION\n" + "="*50)
    
    # Summary Tables
    summary = analyze_experiment_results(results_acc)
    print("\n--- Summary Statistics ---")
    print(summary)
    
    # Plotting
    try:
        fig, axes = plt.subplots(1, 3, figsize=(18, 5))
        
        # Plot 1: Accuracy
        acc_summary = df_acc.groupby(['task_type', 'ut_steps'])['is_correct'].mean().reset_index()
        sns.barplot(data=acc_summary, x='ut_steps', y='is_correct', hue='task_type', ax=axes[0])
        axes[0].set_title('Accuracy by UT Steps')
        axes[0].set_ylabel('Accuracy')
        axes[0].yaxis.set_major_formatter(plt.FuncFormatter(lambda y, _: '{:.0%}'.format(y)))
        
        # Plot 2: Time
        time_summary = df_acc.groupby(['task_type', 'ut_steps'])['generation_time'].mean().reset_index()
        sns.barplot(data=time_summary, x='ut_steps', y='generation_time', hue='task_type', ax=axes[1])
        axes[1].set_title('Inference Time (s) by UT Steps')
        
        # Plot 3: Token Count
        sns.boxplot(data=df_acc, x='ut_steps', y='generated_tokens', hue='task_type', ax=axes[2])
        axes[2].set_title('Generated Tokens Distribution')
        
        plt.tight_layout()
        plt.show()
        
    except Exception as e:
        print(f"⚠️ Visualization error: {e}")
else:
    print("⚠️ No results to visualize.")

print("\n🏁 Experiment Complete.\n")

NameError: name 'timestamp' is not defined

In [ ]:
print("Final Inspection:\n")
print("Top 20 Accuracy Report:\n")
print(df_acc.head(20))
print(f"Full Response:\n")
print(df_acc['full_response'])
print("Perplexity Report:\n")
print(df_ppl.head(20))

In [4]:
print(df[['full_response', 'generated_tokens']])

                                        full_response  generated_tokens
0    Current: 0\n\na) 645 + 632 = 1277\nb) 1277 + ...              1024
1    Current: 0\n\na) 0000000000000000000000000000...              1024
2    Current: 004 + 982 = 986\n\n- 986 + 674 = 166...                58
3    Current: 0\n\na) 594 + 339 = 933\nb) 933 + 59...               153
4    Current: 0\n\na) 121 + 740 = 861\nb) 861 + 29...               114
5    Current: 0\n\na) 417 + 105 = 522\nb) 522 + 42...               110
6    Current: 0\n\na) 930 + 363 = 1293\nb) 1293 + ...              1024
7    Current: 059 + 478 = 537\n\nStep 1: 537 + 182...                68
8    Current: 0\n\na) 0000000000000000000000000000...              1024
9    Current: 0\n\na) 646 + 609 = 1255\nb) 1255 + ...               157
10   Current: 0\n\na) 286 + 208 = 494\nb) 494 + 42...              1024
11   Current: 0\n\na) 910 + 323 = 1233\nb) 1233 + ...               688
12   Current: 0\n\na) 961 + 452 = 1413\nb) 1413 + ...           

In [7]:
!pip freeze > requirements.txt

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
